In [3]:
import os,sys
import pandas as pd
import numpy as np
from tqdm import tqdm
import subprocess
data_path = os.path.join('..','data')

import warnings
warnings.filterwarnings('ignore')

sys.path.append(os.path.join('..','scripts'))
from ras_method import ras_method

## First iteration, no trade to determine total regional input and output

### Load sector data

In [22]:
sectors = list(pd.read_excel(os.path.join(data_path,'industry_high_level_classification.xlsx'))['SEC_CODE'].values)

### Load provincial data

In [56]:
reg_mapper = pd.read_excel(os.path.join(data_path,'INDEC','sh_cou_06_16.xls'),sheet_name='reg_mapper',header=None).iloc[:,:2]
reg_mapper = dict(zip(reg_mapper[0],reg_mapper[1]))

In [57]:
prov_data = pd.read_excel(os.path.join(data_path,'INDEC','PIB_provincial_06_17.xls'),sheet_name='VBP',
                         skiprows=3,index_col=[0],header=[0],nrows=71)
prov_data = prov_data.loc[[x.isupper() for x in prov_data.index],:]
prov_data.columns = [x.replace(' ','_') for x in ['Ciudad de Buenos Aires', 'Buenos Aires', 'Catamarca', 'Cordoba',
       'Corrientes', 'Chaco', 'Chubut', 'Entre Rios', 'Formosa', 'Jujuy',
       'La Pampa', 'La Rioja', 'Mendoza', 'Misiones', 'Neuquen', 'Rio Negro',
       'Salta', 'San Juan', 'San Luis', 'Santa Cruz', 'Santa Fe',
       'Santiago del Estero', 'Tucuman', 'Tierra del Fuego',
       'No distribuido', 'Total']]
region_names = list(prov_data.columns)[:-2]

In [58]:
prov_data.index = sectors+['TOTAL']

In [59]:
prov_data = prov_data.replace(0, 1)

### Create proxy data for first iteration

In [60]:
sectors+['other1','other2']

['A',
 'B',
 'C',
 'D',
 'E',
 'F',
 'G',
 'H',
 'I',
 'J',
 'K',
 'L',
 'M',
 'N',
 'O',
 'P',
 'other1',
 'other2']

In [61]:
# proxy level 2
proxy_reg_arg = pd.DataFrame(prov_data.iloc[-1,:24]/prov_data.iloc[-1,:24].sum()).reset_index()
proxy_reg_arg['year'] = 2016
proxy_reg_arg = proxy_reg_arg[['year','index','TOTAL']]
proxy_reg_arg.columns = ['year','id','gdp']
proxy_reg_arg.to_csv(os.path.join('..','mrio_downscaling','proxy_reg_arg.csv'),index=False)

In [62]:
# proxy level 4
for iter_,sector in enumerate(sectors+['other1','other2']):
    if (sector is not 'other1') & (sector is not 'other2'):
        proxy_sector = pd.DataFrame(prov_data.iloc[iter_,:24]/prov_data.iloc[iter_,:24].sum()).reset_index()
        proxy_sector['year'] = 2016
        proxy_sector['sector'] = 'sec{}'.format(sector)
        proxy_sector = proxy_sector[['year','sector','index',sector]]
        proxy_sector.columns = ['year','sector','region','gdp']
        proxy_sector.to_csv(os.path.join('..','mrio_downscaling','proxy_sec{}.csv'.format(sector)),index=False)
    else:
        proxy_sector = pd.DataFrame(prov_data.iloc[-1,:24]/prov_data.iloc[-1,:24].sum()).reset_index()
        proxy_sector['year'] = 2016
        proxy_sector['sector'] = sector+'1'
        proxy_sector = proxy_sector[['year','sector','index','TOTAL']]
        proxy_sector.columns = ['year','sector','region','gdp']
        proxy_sector.to_csv(os.path.join('..','mrio_downscaling','proxy_{}.csv'.format(sector)),index=False)

In [63]:
def change_name(x):
    if x in sectors:
        return 'sec'+x
    elif x == 'other1':
        return 'other11'
    else:
        return 'other21'

In [64]:
# proxy level 18
mi_index = pd.MultiIndex.from_product([sectors+['other1','other2'], region_names, sectors+['other1','other2'], region_names],
                                     names=['sec1', 'reg1','sec2','reg2'])
for iter_,sector in enumerate(sectors+['other1','other2']):
    if (sector is not 'other1') & (sector is not 'other2'):
        proxy_trade = pd.DataFrame(columns=['year','gdp'],index= mi_index).reset_index()
        proxy_trade['year'] = 2016
        proxy_trade['gdp'] = 0
        proxy_trade = proxy_trade.query("reg1 != reg2")
        proxy_trade = proxy_trade.loc[proxy_trade.sec1 == sector]
        proxy_trade['sec1'] = proxy_trade.sec1.apply(change_name)
        proxy_trade['sec2'] = proxy_trade.sec2.apply(change_name)
        proxy_trade = proxy_trade[['year','sec1','reg1','sec2','reg2','gdp']]
        proxy_trade.columns = ['year','sector','region','sector','region','gdp']
        proxy_trade.to_csv(os.path.join('..','mrio_downscaling','proxy_trade_sec{}.csv'.format(sector)),index=False)    
    else:
        proxy_trade = pd.DataFrame(columns=['year','gdp'],index= mi_index).reset_index()
        proxy_trade['year'] = 2016
        proxy_trade['gdp'] = 0
        proxy_trade = proxy_trade.query("reg1 != reg2")    
        proxy_trade = proxy_trade.loc[proxy_trade.sec1 == sector]
        proxy_trade['sec1'] = proxy_trade.sec1.apply(change_name)
        proxy_trade['sec2'] = proxy_trade.sec2.apply(change_name)       
        proxy_trade = proxy_trade[['year','sec1','reg1','sec2','reg2','gdp']]
        proxy_trade.columns = ['year','sector','region','sector','region','gdp']
        proxy_trade.to_csv(os.path.join('..','mrio_downscaling','proxy_trade_{}.csv'.format(sector)),index=False)

### Create first version of MRIO for Argentina

In [65]:
p = subprocess.Popen(['..\mrio_downscaling\mrio_disaggregate', 'settings_notrade.yml'],
                     cwd=os.path.join('..','mrio_downscaling'))
p.wait()

0

In [66]:
region_names_list = [item for sublist in [[x]*(len(sectors)+2) for x in region_names]
                     for item in sublist]

In [67]:
rows = ([x for x in sectors+['VA','IMP']])*len(region_names)
cols = ([x for x in sectors+['FD','EXP']])*len(region_names)

In [68]:
index_mi = pd.MultiIndex.from_arrays([region_names_list, rows], names=('region', 'row'))
column_mi = pd.MultiIndex.from_arrays([region_names_list, cols], names=('region', 'col'))

MRIO = pd.read_csv(os.path.join('..','mrio_downscaling','output1.csv'),header=None,index_col=None)
MRIO.index = index_mi
MRIO.columns = column_mi

# create predefined index and col, which is easier to read
sector_only = [x for x in sectors]*len(region_names)
col_only = ['FD']*len(region_names)

region_col = [item for sublist in [[x]*len(sectors) for x in region_names] for item in sublist] + \
    [item for sublist in [[x]*1 for x in region_names] for item in sublist]

column_mi_reorder = pd.MultiIndex.from_arrays(
    [region_col, sector_only+col_only], names=('region', 'col'))

# sum va and imports
valueA = MRIO.xs('VA', level=1, axis=0).sum(axis=0)
valueA.drop('FD', level=1,axis=0,inplace=True)
valueA.drop('EXP', level=1,axis=0,inplace=True)
imports = MRIO.xs('IMP', level=1, axis=0).sum(axis=0)
imports.drop('FD', level=1,axis=0,inplace=True)
imports.drop('EXP', level=1,axis=0,inplace=True)
FinalD = MRIO.xs('FD', level=1, axis=1).sum(axis=1)
FinalD.drop('VA', level=1,axis=0,inplace=True)
FinalD.drop('IMP', level=1,axis=0,inplace=True)
Export = MRIO.xs('EXP', level=1, axis=1).sum(axis=1)
Export.drop('VA', level=1,axis=0,inplace=True)
Export.drop('IMP', level=1,axis=0,inplace=True)

output_new = MRIO.copy()

#MRIO.drop('FD', level=1,axis=1,inplace=True)
#MRIO.drop('EXP', level=1,axis=1,inplace=True)
#MRIO.drop('VA', level=1,axis=0,inplace=True)
#MRIO.drop('IMP', level=1,axis=0,inplace=True)

#output_new = pd.concat([MRIO,pd.DataFrame(FinalD,columns=[('ALL','FD')]),
#                        pd.DataFrame(Export,columns=[('ALL','EXP')])],axis=1)
#output_new = pd.concat([MRIO,pd.concat([pd.DataFrame(FinalD,columns=[('ALL','VA')]),
#                           pd.DataFrame(Export,columns=[('ALL','IMP')])],axis=1).T])


In [69]:
%%time
# convert to numpy matrix
X0 = MRIO.as_matrix()

# get sum of rows and columns
u = X0.sum(axis=1)
v = X0.sum(axis=0)

# and only keep T
v[:(len(u)-2)] = u[:-2]

# apply RAS method to rebalance the table
X1 = ras_method(X0, u, v, eps=1e-5,print_out=True)

0.6008773628165098
0.2625041983291241
0.1546234605495287
0.09446193578605078
0.05682675959372796
0.034110900684548584
0.020530782741094766
0.012413610231906791
0.007544043851130722
0.0046077224962706165
0.002827353194470472
0.0017420809853030406
0.001077261041797728
0.0006999152316877755
0.00047958874351827774
0.0003276717655409378
0.000223372620663298
0.0001520032127895199
0.00010329304644063342
7.011524499889177e-05
4.755287766000649e-05
3.2228692425473326e-05
2.1830919487708655e-05
1.4781337931424332e-05
1.000475694112879e-05
Wall time: 582 ms


In [70]:
output_new = pd.DataFrame(X1)
output_new.index = index_mi
output_new.columns = column_mi

In [89]:
output_new

region                     Ciudad_de_Buenos_Aires                      \
col                                             A         B         C   
region                 row                                              
Ciudad_de_Buenos_Aires A                 0.002991  0.000111  0.000341   
                       B                 0.000391  0.000057  0.002647   
                       C                 0.000237  0.000046  0.120775   
                       D                 0.133751  0.040704  6.283528   
                       E                 0.005512  0.000253  0.556652   
...                                           ...       ...       ...   
Tierra_del_Fuego       N                 0.000000  0.000000  0.000000   
                       O                 0.000000  0.000000  0.000000   
                       P                 0.000000  0.000000  0.000000   
                       VA                0.000000  0.000000  0.000000   
                       IMP               0.000000  0.000000  0.000000   

region                                                                 \
col                                 D         E          F          G   
region                 row                                              
Ciudad_de_Buenos_Aires A     0.652041  0.000343   0.000595   0.003727   
                       B     0.264516  0.002964   0.056529   0.033796   
                       C    24.751913  2.060684   9.909002   0.123184   
                       D    45.370659  1.626190  18.935769  10.551072   
                       E     3.031376  3.300959   0.513450   2.490020   
...                               ...       ...        ...        ...   
Tierra_del_Fuego       N     0.000000  0.000000   0.000000   0.000000   
                       O     0.000000  0.000000   0.000000   0.000000   
                       P     0.000000  0.000000   0.000000   0.000000   
                       VA    0.000000  0.000000   0.000000   0.000000   
                       IMP   0.000000  0.000000   0.000000   0.000000   

region                                                      ...  \
col                                 H          I         J  ...   
region                 row                                  ...   
Ciudad_de_Buenos_Aires A     0.673836   0.000905  0.000802  ...   
                       B     0.084181   0.051237  0.016106  ...   
                       C     0.204613   0.116494  0.051669  ...   
                       D    26.414299  16.172183  5.480211  ...   
                       E     2.083785   1.737825  0.644062  ...   
...                               ...        ...       ...  ...   
Tierra_del_Fuego       N     0.000000   0.000000  0.000000  ...   
                       O     0.000000   0.000000  0.000000  ...   
                       P     0.000000   0.000000  0.000000  ...   
                       VA    0.000000   0.000000  0.000000  ...   
                       IMP   0.000000   0.000000  0.000000  ...   

region                     Tierra_del_Fuego                                \
col                                       I         J         K         L   
region                 row                                                  
Ciudad_de_Buenos_Aires A           0.000000  0.000000  0.000000  0.000000   
                       B           0.000000  0.000000  0.000000  0.000000   
                       C           0.000000  0.000000  0.000000  0.000000   
                       D           0.000000  0.000000  0.000000  0.000000   
                       E           0.000000  0.000000  0.000000  0.000000   
...                                     ...       ...       ...       ...   
Tierra_del_Fuego       N           0.000376  0.000450  0.000274  0.000590   
                       O           0.001398  0.000130  0.000100  0.000590   
                       P           0.000363  0.000041  0.000029  0.000156   
                       VA          0.192849  0.106574  0.189294  0.220563   
              

In [71]:
output_new.sum(axis=0).loc[:,'EXP'].sum()

723.1639912728803

## Second iteration, including trade

### Load od matrix

In [72]:
od_matrix_total = pd.DataFrame(pd.read_excel(os.path.join(data_path,'OD_data','province_ods.xlsx'),
                          sheet_name='total',index_col=[0,1],usecols =[0,1,2,3,4,5,6,7])).unstack(1).fillna(0)
od_matrix_total.columns.set_levels(['A','G','C','D','B','I'],level=0,inplace=True)
od_matrix_total.index = od_matrix_total.index.map(reg_mapper)
od_matrix_total = od_matrix_total.stack(0)
od_matrix_total.columns = od_matrix_total.columns.map(reg_mapper)
od_matrix_total = od_matrix_total.swaplevel(i=-2, j=-1, axis=0)
od_matrix_total = od_matrix_total.loc[:, od_matrix_total.columns.notnull()]

In [73]:
def est_trade_value(x,output_new,sector):
    if (sector is not 'other1') & (sector is not 'other2'):
        sec_output = output_new.sum(axis=1).loc[output_new.sum(axis=1).index.get_level_values(1) == sector].reset_index()
    else:
        sec_output = output_new.sum(axis=1).loc[output_new.sum(axis=1).index.get_level_values(1) == 'IMP'].reset_index()
    x['gdp'] = x.gdp*min(sec_output.loc[sec_output.region==x.reg1].values[0][2],sec_output.loc[sec_output.region==x.reg2].values[0][2])
#    x['gdp'] = x.gdp*(sec_output.loc[sec_output.region==x.reg1].values[0][2])
    return x

### Create proxy data for second iteration

In [75]:
# proxy level 14 
mi_index = pd.MultiIndex.from_product([sectors+['other1','other2'], region_names, region_names],
                                     names=['sec1', 'reg1','reg2'])

for iter_,sector in enumerate(tqdm(sectors+['other1','other2'])):
#    if sector in ['L','M','N','O','P']:
#        proxy_trade = pd.DataFrame(columns=['year','gdp'],index= mi_index).reset_index()
#        proxy_trade['year'] = 2016
#        proxy_trade['gdp'] = 0
#        proxy_trade = proxy_trade.query("reg1 != reg2")
#        proxy_trade = proxy_trade.loc[proxy_trade.sec1 == sector]
#        proxy_trade['sec1'] = ['sec'+x if x is not 'other' else 'other1' for x in proxy_trade.sec1 ]
#        proxy_trade = proxy_trade[['year','sec1','reg1','reg2','gdp']]
#        proxy_trade.columns = ['year','sector','region','region','gdp']
#        proxy_trade.to_csv(os.path.join(data_path,'mrio_analysis','proxy_trade_sec{}.csv'.format(sector)),index=False)
    if sector in ['A','G','C','D','B','I']:
        proxy_trade = (od_matrix_total.sum(level=1).divide(od_matrix_total.sum(level=1).sum(axis=1),axis='rows')).stack(0).reset_index()
        proxy_trade.columns = ['reg1','reg2','gdp']
        proxy_trade['year'] = 2016
        proxy_trade = proxy_trade.apply(lambda x: est_trade_value(x,output_new,sector),axis=1)
        proxy_trade['sec1'] = 'sec{}'.format(sector)
        proxy_trade = proxy_trade[['year','sec1','reg1','reg2','gdp']]
        proxy_trade.columns = ['year','sector','region','region','gdp']
        proxy_trade.to_csv(os.path.join('..','mrio_downscaling','proxy_trade14_sec{}.csv'.format(sector)),index=False)
    elif (sector is not 'other1') &  (sector is not 'other2') & (sector not in ['A','G','C','D','B','I']): # &  (sector not in ['L','M','N','O','P']):
        proxy_trade = (od_matrix_total.sum(level=1).divide(od_matrix_total.sum(level=1).sum(axis=1),axis='rows')).stack(0).reset_index()
        #proxy_trade[0].loc[(proxy_trade.origin_province == proxy_trade.destination_province)] = 0.9
        #proxy_trade[0].loc[~(proxy_trade.origin_province == proxy_trade.destination_province)] = 0.1
        proxy_trade.columns = ['reg1','reg2','gdp']
        proxy_trade['year'] = 2016
        proxy_trade = proxy_trade.apply(lambda x: est_trade_value(x,output_new,sector),axis=1)
        proxy_trade['sec1'] = 'sec{}'.format(sector)
        proxy_trade = proxy_trade[['year','sec1','reg1','reg2','gdp']]
        proxy_trade.columns = ['year','sector','region','region','gdp']
        proxy_trade.to_csv(os.path.join('..','mrio_downscaling','proxy_trade14_sec{}.csv'.format(sector)),index=False)

    else:
        proxy_trade = (od_matrix_total.sum(level=1).divide(od_matrix_total.sum(level=1).sum(axis=1),axis='rows')).stack(0).reset_index()
        proxy_trade.columns = ['reg1','reg2','gdp']
        proxy_trade['year'] = 2016
        proxy_trade = proxy_trade.apply(lambda x: est_trade_value(x,output_new,sector),axis=1)
        proxy_trade['sec1'] = sector+'1'
        proxy_trade = proxy_trade[['year','sec1','reg1','reg2','gdp']]
        proxy_trade.columns = ['year','sector','region','region','gdp']
        proxy_trade.to_csv(os.path.join('..','mrio_downscaling','proxy_trade14_{}.csv'.format(sector)),index=False)       

100%|██████████| 18/18 [02:11<00:00,  7.33s/it]


In [193]:
# proxy level 18
mi_index = pd.MultiIndex.from_product([sectors+['other1','other2'], region_names, sectors+['other1','other2'], region_names],
                                     names=['sec1', 'reg1','sec2','reg2'])
for iter_,sector in enumerate(tqdm(sectors+['other1','other2'])):
    if (sector is not 'other1') & (sector is not 'other2'):
        proxy_trade = pd.DataFrame(columns=['year','gdp'],index= mi_index).reset_index()
        proxy_trade['year'] = 2016
        proxy_trade['gdp'] = 0
        proxy_trade = proxy_trade.query("reg1 != reg2")
        proxy_trade = proxy_trade.loc[proxy_trade.sec1 == sector]
        proxy_trade = proxy_trade.loc[proxy_trade.sec2.isin(['L','M','N','O','P'])]
        proxy_trade['sec1'] = proxy_trade.sec1.apply(change_name)
        proxy_trade['sec2'] = proxy_trade.sec2.apply(change_name) 
        
        proxy_trade = proxy_trade.query("reg1 == reg2")    

        proxy_trade = proxy_trade[['year','sec1','reg1','sec2','reg2','gdp']]
        proxy_trade.columns = ['year','sector','region','sector','region','gdp']
        proxy_trade.to_csv(os.path.join(data_path,'mrio_analysis','proxy_trade_sec{}.csv'.format(sector)),index=False)
    
    else:
        proxy_trade = pd.DataFrame(columns=['year','gdp'],index= mi_index).reset_index()
        proxy_trade['year'] = 2016
        proxy_trade['gdp'] = 0
        proxy_trade = proxy_trade.query("reg1 != reg2")    
        proxy_trade = proxy_trade.loc[proxy_trade.sec1 == sector]
        proxy_trade = proxy_trade.loc[proxy_trade.sec2.isin(['L','M','N','O','P'])]
        proxy_trade['sec1'] = proxy_trade.sec1.apply(change_name)
        proxy_trade['sec2'] = proxy_trade.sec2.apply(change_name) 
        
        proxy_trade = proxy_trade.query("reg1 == reg2")    

        proxy_trade = proxy_trade[['year','sec1','reg1','sec2','reg2','gdp']]
        proxy_trade.columns = ['year','sector','region','sector','region','gdp']

        proxy_trade.to_csv(os.path.join(data_path,'mrio_analysis','proxy_trade_{}.csv'.format(sector)),index=False)

100%|██████████████████████████████████████████████████████████████████████████████████| 18/18 [00:02<00:00,  7.16it/s]


In [79]:
%%time
p = subprocess.Popen(['..\mrio_downscaling\mrio_disaggregate', 'settings_trade.yml'],
                     cwd=os.path.join('..','mrio_downscaling'))
p.wait()

Wall time: 54.9 s


0

In [80]:
region_names_list = [item for sublist in [[x]*(len(sectors)+2) for x in region_names]
                     for item in sublist]

rows = ([x for x in sectors+['VA','IMP']])*len(region_names)
cols = ([x for x in sectors+['FD','EXP']])*len(region_names)

index_mi = pd.MultiIndex.from_arrays([region_names_list, rows], names=('region', 'row'))
column_mi = pd.MultiIndex.from_arrays([region_names_list, cols], names=('region', 'col'))

MRIO = pd.read_csv(os.path.join('..','mrio_downscaling','output2.csv'),header=None,index_col=None)
MRIO.index = index_mi
MRIO.columns = column_mi

# create predefined index and col, which is easier to read
sector_only = [x for x in sectors]*len(region_names)
col_only = ['FD','EXP']*len(region_names)

region_col = [item for sublist in [[x]*len(sectors) for x in region_names] for item in sublist] + \
    [item for sublist in [[x]*2 for x in region_names] for item in sublist]

column_mi_reorder = pd.MultiIndex.from_arrays(
    [region_col, sector_only+col_only], names=('region', 'col'))

# sum va and imports
valueA = pd.DataFrame(MRIO.loc[MRIO.index.get_level_values(1) == 'VA'].sum(axis='index'))
valueA.columns = pd.MultiIndex.from_product([['Total'],['ValueA']],names=['region','row'])

IMP = pd.DataFrame(MRIO.loc[MRIO.index.get_level_values(1) == 'IMP'].sum(axis='index'))
IMP.columns = pd.MultiIndex.from_product([['Total'],['IMP']],names=['region','row'])

output = pd.concat([MRIO.loc[~MRIO.index.get_level_values(1).isin(['FD','EXP'])]])
output = output.drop(['VA','IMP'], level=1)
output = pd.concat([output,valueA.T,IMP.T])

output = output.reindex(column_mi_reorder, axis='columns')

In [85]:
output

region                        Ciudad_de_Buenos_Aires                           \
col                                                A             B          C   
region                 row                                                      
Ciudad_de_Buenos_Aires A                    0.965233  2.141120e-03   0.000570   
                       B                    0.009970  3.125630e-04   0.004107   
                       C                    0.002259  3.103570e-04   1.111350   
                       D                    2.868800  7.390630e-02   1.122680   
                       E                    0.093008  4.043210e-04   0.226487   
...                                              ...           ...        ...   
Tierra_del_Fuego       N                    0.000023  1.099060e-07   0.000006   
                       O                    0.000000  0.000000e+00   0.000000   
                       P                    0.000000  0.000000e+00   0.000000   
Total                  ValueA              63.599600  2.371410e+00  37.389700   
                       IMP                  1.542310  5.378680e-01   4.023890   

region                                                                       \
col                                     D          E          F           G   
region                 row                                                    
Ciudad_de_Buenos_Aires A         4.117730   0.001171   0.002407    0.009545   
                       B         0.186353   0.001262   0.025093    0.009341   
                       C         7.434290   0.967847   3.199180    0.023074   
                       D        24.138300   0.495377   6.131270    2.312650   
                       E         1.344770   0.912956   0.143111    0.488258   
...                                   ...        ...        ...         ...   
Tierra_del_Fuego       N         0.000009   0.000002   0.000000    0.000021   
                       O         0.000000   0.000000   0.000000    0.000000   
                       P         0.000000   0.000000   0.000000    0.000000   
Total                  ValueA  165.815000  13.618500  41.379200  155.776000   
                       IMP      91.983000   0.907619   0.496488    5.631110   

region                                                             ...  \
col                                       H          I          J  ...   
region                 row                                         ...   
Ciudad_de_Buenos_Aires A       5.274150e-01   0.002439   0.000572  ...   
                       B       8.385780e-03   0.015968   0.001494  ...   
                       C       2.219910e-02   0.023120   0.005790  ...   
                       D       2.234760e+00   3.903040   0.483191  ...   
                       E       1.797770e-01   0.376582   0.061009  ...   
...                                     ...        ...        ...  ...   
Tierra_del_Fuego       N       7.222690e-07   0.000020   0.000104  ...   
                       O       0.000000e+00   0.000000   0.000000  ...   
                       P       0.000000e+00   0.000000   0.000000  ...   
Total                  ValueA  2.296110e+01  66.460900  38.383200  ...   
                       IMP     1.426570e+01  12.059700   0.504760  ...   

region                        Santa_Cruz           Santa_Fe            \
col                                   FD       EXP       FD       EXP   
region                 row                                              
Ciudad_de_Buenos_Aires A        0.000000  0.000000  0.00000  0.000000   
                       B        0.000000  0.000000  0.00000  0.000000   
                       C        0.000000  0.000000  0.00000  0.000000   
                       D        0.000000  0.000000  0.00000  0.000000   
                       E        0.000000  0.000000  0.00000  0.000000   
...                                  ...       ...      ...       ...   
Tierra_del_Fuego       N        0.015826  0.000002  0.01543  0.0

In [81]:
prov_ratios = pd.DataFrame((prov_data.iloc[:16,:24].stack().swaplevel(i=-2,
             j=-1)/sum(prov_data.iloc[:16,:24].stack().swaplevel(i=-2, 
             j=-1))),columns=['ratio']).reset_index().groupby(['level_0','level_1']).sum().reindex(output.index)[:384]

In [82]:
output_ratio = pd.DataFrame((output.sum(level=0,axis=1).sum(axis=1)[:-2]/sum(output.sum(level=0,axis=1).sum(axis=1)[:-2])),columns=['ratio'])

In [83]:
new_values = pd.DataFrame(output.sum(level=0,axis=1).sum(axis=1)[:-2],columns=['ratio']).multiply(prov_ratios).divide(output_ratio).fillna(0)

In [87]:
output

region                        Ciudad_de_Buenos_Aires                           \
col                                                A             B          C   
region                 row                                                      
Ciudad_de_Buenos_Aires A                    0.965233  2.141120e-03   0.000570   
                       B                    0.009970  3.125630e-04   0.004107   
                       C                    0.002259  3.103570e-04   1.111350   
                       D                    2.868800  7.390630e-02   1.122680   
                       E                    0.093008  4.043210e-04   0.226487   
...                                              ...           ...        ...   
Tierra_del_Fuego       N                    0.000023  1.099060e-07   0.000006   
                       O                    0.000000  0.000000e+00   0.000000   
                       P                    0.000000  0.000000e+00   0.000000   
Total                  ValueA              63.599600  2.371410e+00  37.389700   
                       IMP                  1.542310  5.378680e-01   4.023890   

region                                                                       \
col                                     D          E          F           G   
region                 row                                                    
Ciudad_de_Buenos_Aires A         4.117730   0.001171   0.002407    0.009545   
                       B         0.186353   0.001262   0.025093    0.009341   
                       C         7.434290   0.967847   3.199180    0.023074   
                       D        24.138300   0.495377   6.131270    2.312650   
                       E         1.344770   0.912956   0.143111    0.488258   
...                                   ...        ...        ...         ...   
Tierra_del_Fuego       N         0.000009   0.000002   0.000000    0.000021   
                       O         0.000000   0.000000   0.000000    0.000000   
                       P         0.000000   0.000000   0.000000    0.000000   
Total                  ValueA  165.815000  13.618500  41.379200  155.776000   
                       IMP      91.983000   0.907619   0.496488    5.631110   

region                                                             ...  \
col                                       H          I          J  ...   
region                 row                                         ...   
Ciudad_de_Buenos_Aires A       5.274150e-01   0.002439   0.000572  ...   
                       B       8.385780e-03   0.015968   0.001494  ...   
                       C       2.219910e-02   0.023120   0.005790  ...   
                       D       2.234760e+00   3.903040   0.483191  ...   
                       E       1.797770e-01   0.376582   0.061009  ...   
...                                     ...        ...        ...  ...   
Tierra_del_Fuego       N       7.222690e-07   0.000020   0.000104  ...   
                       O       0.000000e+00   0.000000   0.000000  ...   
                       P       0.000000e+00   0.000000   0.000000  ...   
Total                  ValueA  2.296110e+01  66.460900  38.383200  ...   
                       IMP     1.426570e+01  12.059700   0.504760  ...   

region                        Santa_Cruz           Santa_Fe            \
col                                   FD       EXP       FD       EXP   
region                 row                                              
Ciudad_de_Buenos_Aires A        0.000000  0.000000  0.00000  0.000000   
                       B        0.000000  0.000000  0.00000  0.000000   
                       C        0.000000  0.000000  0.00000  0.000000   
                       D        0.000000  0.000000  0.00000  0.000000   
                       E        0.000000  0.000000  0.00000  0.000000   
...                                  ...       ...      ...       ...   
Tierra_del_Fuego       N        0.015826  0.000002  0.01543  0.0

In [84]:
%%time
#mrio_arg = ras_method(np.array(output).T,np.array(list(new_values.ratio)+list(output.sum(axis=0)[-48:])),
#                      np.array(list(new_values.ratio)+[output.sum(axis=0)[386].sum(),output.sum(axis=0)[387].sum()]), eps=1e-10,print_out=True)
mrio_arg = ras_method(np.array(output).T,np.array(list(new_values.ratio)+list(output.sum(axis=0)[-48:])),
                      np.array(list(new_values.ratio)+[6948,986]), 
                      eps=1e-3,print_out=False)

1.0856760245210496e+126
2.3178911094022997e+249
Wall time: 38.5 s


In [88]:
mrio_arg = ras_method(np.array(output).T,np.array(list(output.sum(axis=1))[:384]+list(output.sum(axis=0)[-48:])),
                      np.array(list(output.sum(axis=1))[:384]+[6948,986]), 
                      eps=1e-4,print_out=False)

2.5618693166046106e+58
1.0816586339080528e+117
4.566920696245695e+175
1.928220604169845e+234
8.141228949742016e+292


In [201]:
mrio_argentina = pd.DataFrame(mrio_arg.T,index=output.index,columns=output.columns)

In [202]:
national = mrio_argentina.sum(level=1,axis=0).sum(level=1,axis=1)

In [152]:
mrio_argentina.to_csv('D:\Dropbox\OIA\Argentina\Data\economic_IO_tables\output\mrio_argentina.csv')